# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,21.60,58,56,5.12,AU,1699842054
1,1,carnarvon,-24.8667,113.6333,23.19,66,27,4.52,AU,1699842059
2,2,chebba,35.2372,11.1150,18.84,76,9,6.33,TN,1699842182
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,-0.67,74,40,12.52,RU,1699842067
4,4,george town,5.4112,100.3354,31.64,75,20,1.54,MY,1699842044


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    color='City'
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_df = ideal_df.dropna(how='any')

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,port mathurin,-19.6833,63.4167,25.06,78,0,2.16,MU,1699842190
74,74,tura,25.5198,90.2201,21.66,72,0,0.91,IN,1699842192
94,94,nsanje,-16.9200,35.2620,23.37,58,0,1.80,MW,1699842195
213,213,vila velha,-20.3297,-40.2925,26.08,84,0,3.60,BR,1699842080
227,227,sarh,9.1429,18.3923,23.95,94,0,0.00,TD,1699842215
255,255,bom jesus,-9.0744,-44.3586,25.27,36,0,0.89,BR,1699842189
286,286,sirte,31.2089,16.5887,21.14,68,0,2.22,LY,1699842222
304,304,tshabong,-26.0500,22.4500,21.27,16,0,3.78,BW,1699842054
365,365,praia,14.9215,-23.5087,24.30,83,0,4.12,CV,1699842233
383,383,san ignacio,17.1561,-89.0714,24.08,94,0,0.00,BZ,1699842235


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City','Country','Lat','Lng','Humidity']].copy(deep=False)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
60,port mathurin,MU,-19.6833,63.4167,78,
74,tura,IN,25.5198,90.2201,72,
94,nsanje,MW,-16.9200,35.2620,58,
213,vila velha,BR,-20.3297,-40.2925,84,
227,sarh,TD,9.1429,18.3923,94,
255,bom jesus,BR,-9.0744,-44.3586,36,
286,sirte,LY,31.2089,16.5887,68,
304,tshabong,BW,-26.0500,22.4500,16,
365,praia,CV,14.9215,-23.5087,83,
383,san ignacio,BZ,17.1561,-89.0714,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'limit':1,
    'apiKey': geoapify_key,
    'categories': f'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = hotel_df.loc[index, 'Lat']
    Lng = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{Lng},{Lat},{radius}'
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port mathurin - nearest hotel: Escale Vacances
tura - nearest hotel: No hotel found
nsanje - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
sarh - nearest hotel: فندق سفاري
bom jesus - nearest hotel: Hotel Brasão
sirte - nearest hotel: فندق المدينة-سرت
tshabong - nearest hotel: No hotel found
praia - nearest hotel: Casa Sodadi
san ignacio - nearest hotel: Rainforest Haven Inn
misratah - nearest hotel: الفندق السياحى
tezu - nearest hotel: No hotel found
carutapera - nearest hotel: No hotel found
ambovombe - nearest hotel: La source
potam - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
60,port mathurin,MU,-19.6833,63.4167,78,Escale Vacances
74,tura,IN,25.5198,90.2201,72,No hotel found
94,nsanje,MW,-16.9200,35.2620,58,No hotel found
213,vila velha,BR,-20.3297,-40.2925,84,Hotel Prainha
227,sarh,TD,9.1429,18.3923,94,فندق سفاري
255,bom jesus,BR,-9.0744,-44.3586,36,Hotel Brasão
286,sirte,LY,31.2089,16.5887,68,فندق المدينة-سرت
304,tshabong,BW,-26.0500,22.4500,16,No hotel found
365,praia,CV,14.9215,-23.5087,83,Casa Sodadi
383,san ignacio,BZ,17.1561,-89.0714,94,Rainforest Haven Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    hover_cols=['Hotel Name','Country'],
    geo=True,
    tiles='OSM',
    color='City',
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)